#### Dependencies
____

In [14]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

#### Creation Session
____

In [2]:
spark = SparkSession.builder.appName('lr3').getOrCreate()

#### Load Data
___

In [3]:
data = spark.read.csv('resources/titanic.csv',inferSchema=True, header=True)

data.show(n=5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### Features selection
___________

In [4]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
columns = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

#### Missing Data
___________

In [6]:
final_data = data.select(columns).na.drop()

#### Categorical Columns
___________

In [11]:
gender_index = StringIndexer(inputCol='Sex', outputCol='SexIndex')
embark_index = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')

#### OneHotEncoder
___________

In [12]:
# Example
# STR : ABC
# KEY : B
# OUTPUT : [0,1,0]

gender_enconder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')
embark_enconder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

#### Assembler 
____

In [13]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkedVec', 'Age', 'SibSp', 'Parch', 'Fare'], outputCol='features')

#### Model & Pipeline
_____

In [16]:
model = LogisticRegression(featuresCol='features', labelCol='Survived')

pipeline = Pipeline(stages=[gender_index, embark_index, gender_enconder, embark_enconder, assembler, model])

#### Split Training & Test Data
_____

In [17]:
train, test = final_data.randomSplit([0.7, 0.3])

#### Fit training data into Pipeline
_____

In [19]:
fit_model = pipeline.fit(train)

#### Evaluate Model 
_____

In [23]:
results = fit_model.transform(test)
evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

AUC = evaluation.evaluate(results)

print (AUC)

0.7506638550452983
